# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [88]:
import pandas as pd
import sqlite3
# from sklearn.ensemble import RandomForestClassifier
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.core.display import Image, display
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
df = pd.read_csv('/content/drive/MyDrive/ex09_42/dayofweek-not-scaled.csv')
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,6,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,7,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,8,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [79]:
y = df['dayofweek']
X = df.drop('dayofweek', axis=1)
X

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,6,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,7,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,8,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [80]:
svc = svm.SVC(probability=True, random_state=21)
parameters_0 = {'kernel':('linear', 'rbf','sigmoid'), 'C':(0.01, 0.1, 1, 1.5, 5, 10), 'gamma':('scale', 'auto'), 'class_weight':('balanced', 'None')}
clf = GridSearchCV(svc, parameters_0)

In [49]:
clf.fit(X, y)

KeyboardInterrupt: ignored

In [19]:
clf.best_params_

{'C': 10, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear'}

In [20]:
clf.best_score_

0.45026776464804313

In [47]:
result = pd.DataFrame(clf.cv_results_)
result = result.sort_values('rank_test_score', ascending=False)
result

AttributeError: ignored

## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [50]:
svc = DecisionTreeClassifier(random_state=21)

In [81]:
parameters_1 = {'max_depth':(np.arange(1, 49)), 'class_weight':('balanced', None), 'criterion':('entropy', 'gini')}
clf = GridSearchCV(svc, parameters_1)

In [52]:
clf.fit(X, y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=21),
             param_grid={'class_weight': ('balanced', None),
                         'criterion': ('entropy', 'gini'),
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])})

In [53]:
clf.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 16}

In [54]:
clf.best_score_

0.4805014661211876

In [62]:
result = pd.DataFrame(clf.cv_results_)
result = result.sort_values('rank_test_score', ascending=False)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
48,0.003939,0.000041,0.001832,0.000018,balanced,gini,1,"{'class_weight': 'balanced', 'criterion': 'gin...",0.165680,0.106825,0.320475,0.284866,0.130564,0.201682,0.085303,192
0,0.004983,0.001341,0.001915,0.000095,balanced,entropy,1,"{'class_weight': 'balanced', 'criterion': 'ent...",0.165680,0.299703,0.320475,0.284866,0.130564,0.240258,0.076880,191
1,0.004301,0.000070,0.001758,0.000040,balanced,entropy,2,"{'class_weight': 'balanced', 'criterion': 'ent...",0.310651,0.373887,0.278932,0.189911,0.234421,0.277560,0.063118,190
52,0.005287,0.000086,0.001881,0.000028,balanced,gini,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.254438,0.267062,0.474777,0.284866,0.178042,0.291837,0.098466,189
49,0.004411,0.000271,0.001826,0.000043,balanced,gini,2,"{'class_weight': 'balanced', 'criterion': 'gin...",0.310651,0.183976,0.397626,0.344214,0.234421,0.294178,0.076409,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0.006456,0.000259,0.001926,0.000149,None,gini,20,"{'class_weight': None, 'criterion': 'gini', 'm...",0.352071,0.519288,0.495549,0.605341,0.391691,0.472788,0.090994,5
156,0.005835,0.000202,0.001862,0.000074,None,gini,13,"{'class_weight': None, 'criterion': 'gini', 'm...",0.372781,0.548961,0.543027,0.522255,0.403561,0.478117,0.074612,4
104,0.006969,0.000195,0.002078,0.000082,None,entropy,9,"{'class_weight': None, 'criterion': 'entropy',...",0.316568,0.548961,0.548961,0.635015,0.350148,0.479931,0.124187,3
158,0.006079,0.000163,0.001877,0.000107,None,gini,15,"{'class_weight': None, 'criterion': 'gini', 'm...",0.363905,0.522255,0.551929,0.575668,0.388724,0.480496,0.087086,2


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [67]:
svc = RandomForestClassifier(random_state=21)

In [82]:
parameters_2 = {'n_estimators':(5, 10, 50, 100), 'max_depth':(range(1,49)), 'class_weight':('balanced', 'None'), 'criterion':('entropy', 'gini')}
clf = GridSearchCV(svc, parameters_2)

In [69]:
clf.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1920 fits failed out of a total of 3840.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1920 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 762, in _validate_y_class_weight
    % self.class_weight
ValueError: Val

GridSearchCV(estimator=RandomForestClassifier(random_state=21),
             param_grid={'class_weight': ('balanced', 'None'),
                         'criterion': ('entropy', 'gini'),
                         'max_depth': range(1, 49),
                         'n_estimators': (5, 10, 50, 100)})

In [70]:
clf.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 34,
 'n_estimators': 50}

In [71]:
clf.best_score_

0.5463750812073112

In [72]:
result = pd.DataFrame(clf.cv_results_)
result = result.sort_values('rank_test_score', ascending=False)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
767,0.002590,0.000020,0.000000,0.000000,None,gini,48,100,"{'class_weight': 'None', 'criterion': 'gini', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,768
573,0.002661,0.000096,0.000000,0.000000,None,entropy,48,10,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,767
503,0.002757,0.000039,0.000000,0.000000,None,entropy,30,100,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,766
504,0.003948,0.000919,0.000000,0.000000,None,entropy,31,5,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,765
505,0.003004,0.000073,0.000000,0.000000,None,entropy,31,10,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.025863,0.000544,0.004266,0.000058,balanced,gini,40,10,"{'class_weight': 'balanced', 'criterion': 'gin...",0.331361,0.640950,0.635015,0.750742,0.362018,0.544017,0.166588,5
317,0.026074,0.000674,0.004226,0.000068,balanced,gini,32,10,"{'class_weight': 'balanced', 'criterion': 'gin...",0.331361,0.637982,0.623145,0.744807,0.385757,0.544610,0.158540,4
330,0.114794,0.003037,0.012200,0.002379,balanced,gini,35,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.357988,0.584570,0.632047,0.771513,0.379822,0.545188,0.156658,3
291,0.224268,0.005449,0.019654,0.001048,balanced,gini,25,100,"{'class_weight': 'balanced', 'criterion': 'gin...",0.375740,0.563798,0.640950,0.795252,0.356083,0.546365,0.165236,2


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [ ]:
list_param = ParameterGrid(parameters_2)
data = []

for params in tqdm(list_param):
    row = params
    estimator = RandomForestClassifier(**params)
    scores = cross_val_score(estimator, X_train, y_train, cv=5, n_jobs=-1)
    row['mean_accuracy'] = np.mean(scores)
    row['std_accuracy'] = np.std(scores)
    data.append(row)

  0%|          | 0/768 [00:00<?, ?it/s]

In [ ]:
data

## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [ ]:
svc = RandomForestClassifier(random_state=21)

In [ ]:
parameters = {'n_estimators':(5, 10, 50, 100), 'max_depth':(range(1,49)), 'class_weight':('balanced', 'None'), 'criterion':('entropy', 'gini')}
clf = GridSearchCV(svc, parameters)

In [ ]:
clf.fit(X, y)